# Contrastive Explanations Method(CEM) applied to Heart dataset


Contrastive Explanation Method, abbreviated as CEM, is a XAI Method which can give local explanations for a black box model. This method is applicable for classification datasets. CEM gives two kinds of explanations: 

Pertinent Positives (PP): For a PP, the method finds the features that should be minimally and sufficiently present (e.g. important pixels in an image) to predict the same class as on the original instance.  PP works similarly to Anchors.


Pertinent Negatives (PN): PN’s on the other hand identify what features should be minimally and necessarily absent from the instance to be explained in order to maintain the original prediction class. The aim of PN’s is not to provide a full set of characteristics that should be absent in the explained instance, but to identify a minimal set of features that is enough to differentiate it from the nearest different class. PN works similarly to Counterfactuals.






In [ ]:
!pip install alibi

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel(40)
tf.compat.v1.disable_v2_behavior() 
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from alibi.explainers import CEM

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

Reading the  dataset:

In [ ]:
dataset = pd.read_csv('../input/heart-disease-cleveland-uci/heart_cleveland_upload.csv')
# To display the top 5 rows
dataset.head(5)

In [ ]:
heart = dataset.copy()

In [ ]:
target = 'condition'
feature_names = list(heart.columns)
feature_names.remove(target)

In [ ]:
y = heart.pop('condition')

In [ ]:
heart = (heart - heart.mean(axis=0)) / heart.std(axis=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(heart, y, test_size=0.2, random_state=33)
x_train=X_train.to_numpy()
x_test=X_test.to_numpy()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Training the model:

In [ ]:
def nn_model():
    x_in = Input(shape=(13,))
    x = Dense(40, activation='relu')(x_in)
    x = Dense(40, activation='relu')(x)
    x_out = Dense(2, activation='softmax')(x)
    nn = Model(inputs=x_in, outputs=x_out)
    nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return nn

In [ ]:
nn = nn_model()
nn.summary()
nn.fit(X_train, y_train, batch_size=64, epochs=500, verbose=0)
nn.save('nn_heart.h5', save_format='h5')

Generating contrastive explaination for pertinent negative:

Consider the second instance of testing data.

In [ ]:
idx = 1
X = x_test[idx].reshape((1,) + x_test[idx].shape)
print('Prediction on instance to be explained: {}'.format([np.argmax(nn.predict(X))]))
print('Prediction probabilities for each class on the instance: {}'.format(nn.predict(X)))

The original prediction class is 0, since it has a greater prediction probability.

In [ ]:
mode = 'PN' 
shape = (1,) + x_train.shape[1:]  
kappa = .2 
            
            
beta = .1  
c_init = 10. 
c_steps = 10
max_iterations = 1000  
feature_range = (x_train.min(axis=0).reshape(shape)-.1,  
                 x_train.max(axis=0).reshape(shape)+.1)  
clip = (-1000.,1000.)  
lr_init = 1e-2  

Here, 

*   mode : 'PN' (Pertinent Negative) or 'PP' (Pertinent Positive)
*   shape : Shape of the current instance. As CEM is applicable for single explanations, we take 1.
*   kappa, beta, gamma, c_init, c_steps are all mathematical terms for calculating loss
*   max_iterations : the total no. of loss optimization steps for each value of c
*   feature_range : global or feature wise minimum and maximum values for the changed instance
*   clip : minimum and maximum gradient values
*   lr_init : initial learning rate 



In [ ]:
lr = load_model('nn_heart.h5')

# initialize CEM explainer and explain instance
cem = CEM(lr, mode, shape, kappa=kappa, beta=beta, feature_range=feature_range,
          max_iterations=max_iterations, c_init=c_init, c_steps=c_steps,
          learning_rate_init=lr_init, clip=clip)
cem.fit(x_train, no_info_type='median')  
explanation = cem.explain(X, verbose=False)

In [ ]:
print('Feature names: {}'.format(feature_names))
print('Original instance: {}'.format(explanation.X))
print('Predicted class: {}'.format([explanation.X_pred]))
print('Pertinent negative: {}'.format(explanation.PN))
print('Predicted class: {}'.format([explanation.PN_pred]))

The above result clearly shows that the pertinent negative method pushes the prediction to get a prediction different from the original prediction which is 0 to 1 in this case.

The CEM values in array which are different from the original one change the prediction class. Some of them are cp, ca, thal. Thus, it can be concluded that changes in these features should necessarily be absent to retain the original prediction as 0 as they are responsible for flipping the prediction class.

Generating pertinent positive:

In [ ]:
mode = 'PP'

In [ ]:
# define model
lr = load_model('nn_heart.h5')

# initialize CEM explainer and explain instance
cem = CEM(lr, mode, shape, kappa=kappa, beta=beta, feature_range=feature_range,
          max_iterations=max_iterations, c_init=c_init, c_steps=c_steps,
          learning_rate_init=lr_init, clip=clip)
cem.fit(x_train, no_info_type='median')
explanation = cem.explain(X, verbose=False)

In [ ]:
print('Original instance: {}'.format(explanation.X))
print('Predicted class: {}'.format([explanation.X_pred]))
print('Pertinent positive: {}'.format(explanation.PP))
print('Predicted class: {}'.format([explanation.PP_pred]))

The above result shows that the predicted class remains same on applying PP. The CEM values generated, close to 0, should be compulsorily and minimally present in order to get the same original class 0 as predicted class.